 Source: http://millionsongdataset.com/sites/default/files/tutorial1.py.txt

In [1]:
# imports
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np

In [25]:
# path to the Million Song Dataset subset (uncompressed)
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_subset_path='MillionSongSubset'
#msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' # sanity check

In [3]:
# path to the Million Song Dataset code
# CHANGE IT TO YOUR LOCAL CONFIGURATION
msd_code_path='MSongsDB'
assert os.path.isdir(msd_code_path),'wrong path' # sanity check

In [4]:
# we add some paths to python so we can import MSD code
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

In [5]:
# imports specific to the MSD
import hdf5_getters as GETTERS

In [6]:
# the following function simply gives us a nice string for
# a time lag in seconds
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

In [7]:
# we define this very useful function to iterate the files
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

In [8]:
# we can now easily count the number of files in the dataset
print('number of song files:',apply_to_all_files(msd_subset_path))

number of song files: 10000


In [10]:
# let's now get all artist names in a set(). One nice property:
# if we enter many times the same artist, only one will be kept.
all_artist_names = set()

In [11]:
# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add( artist_name )
    h5.close()

In [14]:
# let's apply the previous function to all files
# we'll also measure how long it takes
t1 = time.time()
apply_to_all_files(msd_subset_path,func=func_to_get_artist_name)
t2 = time.time()
print('all artist names extracted in:',strtimedelta(t1,t2))

all artist names extracted in: 0:03:10.560690


In [22]:
# check N artist names
N = 10
list(all_artist_names)[:5]

[b'Janet Jackson Featuring Kanye West',
 b'Randy Stonehill',
 b'Scott Matthews',
 b'Estrellas Cubanas',
 b'Ray Pillow']

In [24]:
# or 
# let's see some of the content of 'all_artist_names'
print('found',len(all_artist_names),'unique artist names')
for k in range(5):
    print(list(all_artist_names)[k])

found 4412 unique artist names
b'Janet Jackson Featuring Kanye West'
b'Randy Stonehill'
b'Scott Matthews'
b'Estrellas Cubanas'
b'Ray Pillow'


In [26]:

# this is too long, and the work of listing artist names has already
# been done. Let's redo the same task using an SQLite database.
# We connect to the provided database: track_metadata.db

#conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
#                                    'subset_track_metadata.db'))

OperationalError: unable to open database file